In [1]:
# Instalar as bibliotecas necessárias
!pip install transformers
!pip install scikit-learn

!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [2]:
!rm -rf crawl4ai

!git clone https://github.com/unclecode/crawl4ai.git

Cloning into 'crawl4ai'...
remote: Enumerating objects: 10062, done.
remote: Counting objects: 100% (1944/1944), done.
remote: Compressing objects: 100% (501/501), done.
remote: Total 10062 (delta 1681), reused 1485 (delta 1443), pack-reused 8118 (from 2)
Receiving objects: 100% (10062/10062), 144.23 MiB | 27.82 MiB/s, done.
Resolving deltas: 100% (6422/6422), done.


In [3]:
!ls -R crawl4ai

crawl4ai:
CHANGELOG.md	    LICENSE		     ROADMAP.md
cliff.toml	    MANIFEST.in		     setup.cfg
CODE_OF_CONDUCT.md  MISSION.md		     setup.py
CONTRIBUTORS.md     mkdocs.yml		     SPONSORS.md
crawl4ai	    PROGRESSIVE_CRAWLING.md  test_llm_webhook_feature.py
deploy		    prompts		     tests
docker-compose.yml  pyproject.toml	     test_webhook_implementation.py
Dockerfile	    README-first.md	     uv.lock
docs		    README.md
JOURNAL.md	    requirements.txt

crawl4ai/crawl4ai:
'adaptive_crawler copy.py'	  extraction_strategy.py
 adaptive_crawler.py		  html2text
 async_configs.py		  hub.py
 async_crawler_strategy.back.py   __init__.py
 async_crawler_strategy.py	  install.py
 async_database.py		  js_snippet
 async_dispatcher.py		  legacy
 async_logger.py		  link_preview.py
 async_url_seeder.py		  markdown_generation_strategy.py
 async_webcrawler.py		  migrations.py
 browser_adapter.py		  model_loader.py
 browser_manager.py		  models.py
 browser_profiler.py		  processors
 cache_context.py		  pro

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
import os
import numpy as np

# Definir o nome do modelo
model_name = "microsoft/codebert-base"

print("Carregando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Carregando modelo...")
model = AutoModel.from_pretrained(model_name)

model.eval()

print("Modelo e tokenizer carregados com sucesso!")

Carregando tokenizer...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Carregando modelo...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Modelo e tokenizer carregados com sucesso!


In [5]:
def get_code_embedding(file_path):
    """
    Lê um arquivo de código-fonte e retorna seu embedding
    usando o token [CLS] do CodeBERT.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
        
        if not code.strip():
            return None
            
        # Tokenizar o código
        # Usamos truncation=True para garantir que arquivos longos
        # sejam cortados no tamanho máximo do modelo (512 tokens)
        inputs = tokenizer(
            code, 
            return_tensors="pt", 
            truncation=True, 
            max_length=512, 
            padding="max_length"
        )
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
        
        return np.squeeze(cls_embedding)
        
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")
        return None

In [6]:
# Caminho base para o código-fonte do projeto clonado
repo_path = 'crawl4ai/crawl4ai'

file_embeddings = {}
file_paths = []

print(f"Iniciando análise de embeddings para {repo_path}...")

# Navegar por todos os diretórios e subdiretórios
for root, dirs, files in os.walk(repo_path):
    for file in files:
        # Processar apenas arquivos Python
        if file.endswith('.py'):
            file_path = os.path.join(root, file)
            
            embedding = get_code_embedding(file_path)
            
            # Adicionar apenas se o embedding foi gerado com sucesso
            if embedding is not None:
                print(f"  - Processado: {file_path}")
                file_paths.append(file_path)
                file_embeddings[file_path] = embedding

print(f"\nAnálise concluída. {len(file_embeddings)} arquivos .py processados e vetorizados.")

Iniciando análise de embeddings para crawl4ai/crawl4ai...


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

  - Processado: crawl4ai/crawl4ai/types.py
  - Processado: crawl4ai/crawl4ai/browser_adapter.py
  - Processado: crawl4ai/crawl4ai/async_logger.py
  - Processado: crawl4ai/crawl4ai/content_scraping_strategy.py
  - Processado: crawl4ai/crawl4ai/config.py
  - Processado: crawl4ai/crawl4ai/proxy_strategy.py
  - Processado: crawl4ai/crawl4ai/async_webcrawler.py
  - Processado: crawl4ai/crawl4ai/browser_profiler.py
  - Processado: crawl4ai/crawl4ai/__version__.py
  - Processado: crawl4ai/crawl4ai/docker_client.py
  - Processado: crawl4ai/crawl4ai/async_crawler_strategy.back.py
  - Processado: crawl4ai/crawl4ai/model_loader.py
  - Processado: crawl4ai/crawl4ai/chunking_strategy.py
  - Processado: crawl4ai/crawl4ai/table_extraction.py
  - Processado: crawl4ai/crawl4ai/__init__.py
  - Processado: crawl4ai/crawl4ai/async_url_seeder.py
  - Processado: crawl4ai/crawl4ai/async_crawler_strategy.py
  - Processado: crawl4ai/crawl4ai/adaptive_crawler copy.py
  - Processado: crawl4ai/crawl4ai/ssl_certif

In [7]:
from sklearn.cluster import KMeans

if file_embeddings:
    # Preparar os dados para o clustering
    X = np.array(list(file_embeddings.values()))
    
    n_clusters = 5
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X)
    
    print("\n--- Resultado do Clustering (Inferência de Módulos) ---")
    
    # Mostrar quais arquivos caíram em quais clusters
    for i in range(n_clusters):
        print(f"\nCluster {i} (Módulo {i}):")
        for j, file_path in enumerate(file_paths):
            if clusters[j] == i:
                # Imprime o caminho do arquivo relativo ao repositório
                print(f"  {file_path.replace('crawl4ai/src/', '')}")
else:
    print("Nenhum arquivo .py foi processado. Clustering não pode ser executado.")


--- Resultado do Clustering (Inferência de Módulos) ---

Cluster 0 (Módulo 0):
  crawl4ai/crawl4ai/legacy/version_manager.py
  crawl4ai/crawl4ai/html2text/elements.py
  crawl4ai/crawl4ai/crawlers/amazon_product/crawler.py

Cluster 1 (Módulo 1):
  crawl4ai/crawl4ai/__version__.py
  crawl4ai/crawl4ai/html2text/__main__.py
  crawl4ai/crawl4ai/html2text/_typing.py

Cluster 2 (Módulo 2):
  crawl4ai/crawl4ai/browser_adapter.py
  crawl4ai/crawl4ai/async_logger.py
  crawl4ai/crawl4ai/proxy_strategy.py
  crawl4ai/crawl4ai/browser_profiler.py
  crawl4ai/crawl4ai/docker_client.py
  crawl4ai/crawl4ai/model_loader.py
  crawl4ai/crawl4ai/chunking_strategy.py
  crawl4ai/crawl4ai/table_extraction.py
  crawl4ai/crawl4ai/ssl_certificate.py
  crawl4ai/crawl4ai/content_filter_strategy.py
  crawl4ai/crawl4ai/user_agent_generator.py
  crawl4ai/crawl4ai/async_database.py
  crawl4ai/crawl4ai/hub.py
  crawl4ai/crawl4ai/link_preview.py
  crawl4ai/crawl4ai/cache_context.py
  crawl4ai/crawl4ai/migrations.py
  cr

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import re

# Esta função simples limpa os nomes dos arquivos
# 'async_database.py' -> 'async database'
def clean_filename(filename):
    filename = os.path.basename(filename)
    filename = filename.replace('.py', '')
    filename = filename.replace('_', ' ')
    return filename

print("\n--- Análise de Tópicos por Cluster (Palavras-Chave) ---")

if file_embeddings:
    # Palavras comuns para ignorar
    # Adicionamos 'init' porque todo pacote python tem
    stop_words_list = ['init'] 

    for i in range(n_clusters):
        print(f"\nCluster {i} (Módulo {i}):")
        
        # 1. Coletar os nomes dos arquivos do cluster
        cluster_files = []
        for j, file_path in enumerate(file_paths):
            if clusters[j] == i:
                cluster_files.append(file_path)
        
        # 2. Limpar os nomes para criar um "corpus"
        corpus = [clean_filename(f) for f in cluster_files]
        
        if not corpus:
            print("  (Cluster vazio ou apenas com arquivos __init__.py)")
            continue
            
        # 3. Usar CountVectorizer para contar as palavras
        try:
            vectorizer = CountVectorizer(stop_words=stop_words_list)
            X = vectorizer.fit_transform(corpus)
            
            # 4. Somar a frequência das palavras
            word_freq = X.sum(axis=0).A1
            words = vectorizer.get_feature_names_out()
            
            # 5. Mostrar as 5 palavras mais comuns
            top_words = sorted(zip(words, word_freq), key=lambda x: x[1], reverse=True)
            
            print(f"  Top Tópicos: {[word for word, freq in top_words[:5]]}")
            
            # Opcional: mostrar todos os arquivos novamente
            # for file in corpus:
            #    print(f"    - {file}")

        except ValueError as e:
            # Isso pode acontecer se o cluster só tiver nomes com stopwords
            print(f"  Não foi possível extrair tópicos: {e}")
            
else:
    print("Nenhum embedding foi processado.")


--- Análise de Tópicos por Cluster (Palavras-Chave) ---

Cluster 0 (Módulo 0):
  Top Tópicos: ['crawler', 'elements', 'manager', 'version']

Cluster 1 (Módulo 1):
  Top Tópicos: ['main', 'typing', 'version']

Cluster 2 (Módulo 2):
  Top Tópicos: ['strategy', 'async', 'crawler', 'browser', 'utils']

Cluster 3 (Módulo 3):
  Top Tópicos: ['async', 'cli', 'browser', 'config', 'configs']

Cluster 4 (Módulo 4):
  Top Tópicos: ['crawler', 'strategy', 'async', 'adaptive', 'back']
